In [2]:
from pinger2 import read_binary_folder, filter_samples, get_spike_times, pipeline
from matplotlib import pyplot as plt

In [7]:
%matplotlib qt

2024-08-06 11:06:37.553 Python[26318:103028368] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/bw/4tzf4sn540g21cjq2sgsty4w0000gn/T/org.python.python.savedState


In [3]:
readings = read_binary_folder('./binaries/michaelholdingit/')

In [4]:
FS = 625_000
LOWCUT = 30_000.0
HIGHCUT = 40_000.0

In [5]:
channels = readings.channels

In [6]:
readings_times = readings.times

In [5]:
filtered_samples = filter_samples(readings, LOWCUT, HIGHCUT, FS)

In [8]:
plt.plot(filtered_samples[0])

In [9]:
1/(readings_times[1] - readings_times[0])

624999.9926961892

In [ ]:
times = get_spike_times(readings, LOWCUT, HIGHCUT, FS)

starting
made filter
0 channel
1 channel
2 channel
3 channel
finished get spike times


In [11]:
[len(t) for t in times]

[10, 10, 10, 10]

In [14]:
def match_times(times, dt):
    '''
    Takes a 2d list of timings and groups the times together if they are within dt of each other. If one channel does not have a corresponding time, it is filled with None.
    each channel has the times in sequential order (sorted).
    '''
    groupings = []
    indices = [0] * len(times)
    while True:
        ind_times = [times[i][indices[i]] 
                        if indices[i] < len(times[i])
                        else float('inf') 
                    for i in range(len(times))]
        
        min_time = min(ind_times)
        
        group = [None]*len(times)
        
        for i in range(len(times)):
            if indices[i] == len(times[i]):
                continue
            if abs(times[i][indices[i]] - min_time) < dt:
                group[i] = times[i][indices[i]]
                indices[i] += 1
        groupings.append(group)

        if all([indices[i] == len(times[i]) for i in range(len(times))]):
            break
        
    return groupings
    

In [15]:
gs = match_times(times, 0.005)

In [16]:
def what_comes_first(groupings):
    firsts = []
    for group in groupings:
        min_val = min([x for x in group if x is not None])
        firsts.append(group.index(min_val))
    return firsts
        

In [17]:
match_times(times, 0.005)

[[0.6030656099319458,
  0.6030383706092834,
  0.6030591726303101,
  0.6030623912811279],
 [2.651113510131836, 2.65109920501709, 2.6511056423187256, 2.651108741760254],
 [4.699160099029541, 4.699118614196777, 4.69913911819458, 4.6991424560546875],
 [6.747177600860596, 6.747150421142578, 6.747156620025635, 6.747159957885742],
 [8.795210838317871, 8.795169830322266, 8.795190811157227, 8.79517936706543],
 [10.843249320983887,
  10.843209266662598,
  10.84321403503418,
  10.843217849731445],
 [12.89128303527832,
  12.891257286071777,
  12.891263961791992,
  12.891266822814941],
 [14.939311981201172,
  14.93927001953125,
  14.939276695251465,
  14.939279556274414],
 [16.98734474182129,
  16.987316131591797,
  16.987306594848633,
  16.98732566833496],
 [19.03537940979004,
  19.035337448120117,
  19.035356521606445,
  19.03536033630371]]

In [18]:
print(what_comes_first(match_times(times, 0.005)))

[1, 1, 1, 1, 1, 1, 1, 1, 2, 1]


In [1]:
from pinger2 import spikes_pipeline, PipelineConfig, pipeline, get_first_hydrophones

CENTER_FREQUENCY = 35000

LOWCUT = CENTER_FREQUENCY - 1000
HIGHCUT = CENTER_FREQUENCY + 1000

config = PipelineConfig(LOWCUT, HIGHCUT)
config.noise_floor_stdev_mult = 5.5

spikes = spikes_pipeline('./binaries/michaelholdingit/', config)

In [4]:
spikes

[[<Spike at 0x104a26970>,

In [3]:
print(get_first_hydrophones(spikes, config))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
for spike_arr in spikes:
    for spike in spike_arr:
        print('<spike time="{:.6f}" amp="{:.6f}"> '.format(spike.time, spike.amplitude), end='')
    print()

<spike time="0.603051" amp="0.166969"> <spike time="2.651098" amp="0.417935"> <spike time="4.699232" amp="0.213092"> <spike time="6.747208" amp="0.276851"> <spike time="8.795211" amp="0.207666"> <spike time="10.843251" amp="0.310765"> <spike time="12.891342" amp="0.209022"> <spike time="14.939256" amp="0.219875"> <spike time="16.987301" amp="0.280921"> <spike time="19.035364" amp="0.359602"> 
<spike time="0.603010" amp="0.236719"> <spike time="2.651029" amp="0.249149"> <spike time="4.699062" amp="0.164898"> <spike time="6.747138" amp="0.242243"> <spike time="8.795114" amp="0.181472"> <spike time="10.843110" amp="0.152467"> <spike time="12.891214" amp="0.162135"> <spike time="14.939171" amp="0.213239"> <spike time="16.987217" amp="0.185615"> <spike time="19.035236" amp="0.191140"> 
<spike time="0.603174" amp="0.186961"> <spike time="2.651222" amp="0.170824"> <spike time="4.699112" amp="0.182927"> <spike time="6.747344" amp="0.153344"> <spike time="8.795219" amp="0.211165"> <spike time="

In [10]:
firsts = pipeline('./binaries/michaelholdingit/', config)

In [11]:
firsts

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]